In [ ]:
import panel as pn
from bokeh.plotting import figure
import holoviews as hv
from holoviews.streams import Params
import param
hv.extension('bokeh')
pn.extension()

In [ ]:
def make_volume():
    """This function makes a 3D array of data, a brain scan"""
    from skimage import io
    vol = io.imread(
#         "https://s3.amazonaws.com/assets.datacamp.com/blog_assets/" # Since the file has been downloaded
        "attention-mri.tif")
    return vol


TOOLTIPS = [
    ("x", "$x"),
    ("y", "$y"),
    ("value", "@image"),
]
shape = (157, 189, 68)
volume = make_volume()
inverted = False


In [ ]:
def make():
    """Here we make a figure widget with a 2D image and put it in the layout"""
    global inverted
    p1 = figure(width=400, height=400, tools='hover,wheel_zoom',
                tooltips=TOOLTIPS,
                x_range=[0, 157], y_range=[0, 189])
    if inverted:
        p1.image(image=[volume[:, ::-1, 30]], x=[0], y=[0], dw=[157], dh=[189])
    else:
        p1.image(image=[volume[:, :, 30]], x=[0], y=[0], dw=[157], dh=[189])
    widgets[1] = p1
    inverted = not inverted

In [ ]:
button = pn.widgets.Button(name='Invert')
button.on_click(lambda *_: make())
widgets = pn.Column(button, button)
make()
widgets


In [ ]:
volume.shape

## 1. Using simple Dynamic Map and Params

### Simple Image

In [ ]:
hv.Image(volume[:,:,0])

In [ ]:
def get_brain_image(axes = 'XY',index = 0):
    if axes == 'XY':
        data = volume[:,:,index] 
    elif axes == 'XZ':
        data = volume[:,index,:]
    elif axes == 'YZ': 
        data = volume[index,:,:]
    
    image = hv.Image(data)
    return image

### Dmap for Index

In [ ]:
dmap = hv.DynamicMap(get_brain_image, kdims=['index'])
dmap.redim.range(index=(0,67))

### Dmap for both axes and Index

In [ ]:
axes_list = ['XY','XZ','YZ']
dmap = hv.DynamicMap(get_brain_image, kdims=['axes','index'])
dmap.redim.values(axes = axes_list,index = range(67))

### Brain Explorer

In [ ]:
class BrainExplorer(param.Parameterized):
    
    cmap = param.ObjectSelector(default ='dimgray',objects=hv.plotting.list_cmaps() )
    
    axes_list = ['XY','XZ','YZ']
    axes = param.ObjectSelector(default = 'XY',objects=axes_list)
    
    index = param.Integer(default=10,bounds = (0,max(volume.shape)-1))
    
    @param.depends('axes','index','cmap')
    def get_brain_image(self):
        axes = self.axes
        index = self.index
        cmap = self.cmap
        if axes == 'XY':
            data = volume[:,:,index] 
        elif axes == 'XZ':
            data = volume[:,index,:]
        elif axes == 'YZ':
            data = volume[index,:,:]

        return hv.Image(data).opts(cmap = cmap,
                                   tools = ['hover'],
                                   height = 500,
                                   width = 500,
                                   xlabel = axes[0]+'-Axis', ylabel = axes[1]+'-Axis')

In [ ]:
b = BrainExplorer()
dmap = pn.Row(hv.DynamicMap(b.get_brain_image),b.param)
# dmap

### Dashboard 1

In [ ]:
dashboard = pn.Column(
    pn.Column('# <pre> Brain Explorer',
              pn.Spacer(height=50)),
    pn.Row(
        pn.Row(dmap[0],
               pn.Spacer(width = 50),
               pn.Column(
                   dmap[1][1],
                   pn.Spacer(height = 30),
                   dmap[1][2],
                   pn.Spacer(height = 30),
                   dmap[1][3])
              )
        )
)

In [ ]:
dashboard

-------
-------
------
---

## 2. Using watch/callback mechanism

There is a problem with the above dashboard. Axes `XY`, `XZ`, `YZ` have **68, 189, 157** number of images.
However Index slider above is constant, shows 189 indexes for every axes. For `XY` and `YZ` blank images are shown after index **68** and **157**. This has been solved by creating a custom callback function so that it:
1. Resets index to 0 whenever axis is changed.
2. Resets `end` of index slider according to number of images in that axes.

#### Image

In [ ]:
def get_brain_image(axes,index,cmap):
    if axes == 'XY':
        data = volume[:,:,index] 
    elif axes == 'XZ':
        data = volume[:,index,:]
    else: #YZ
        data = volume[index,:,:]
    return hv.Image(data).options(cmap = cmap,
                                   tools = ['hover'],
                                   height = 500,
                                   width = 500,
                                   xlabel = axes[0]+'-Axis', ylabel = axes[1]+'-Axis')

# get_brain_image('XY',0,'dimgray')

#### Streams

In [ ]:
axes_list = ['XY','XZ','YZ']
axes = pn.widgets.Select(name = 'axes',value = 'XY',options=axes_list)
axes_stream  = Params(axes,['value'],rename={'value': 'axes'})

cmaps = pn.widgets.Select(name = 'Color Map', value = 'CMRmap',options=hv.plotting.list_cmaps() )
cmap_stream = Params(cmaps,['value'],rename={'value': 'cmap'})

index = pn.widgets.IntSlider(name = 'index',end = (volume.shape[2]-1))
index_stream = Params(index,['value'],rename={'value': 'index'})

#### Callback Function

In [ ]:
def reset_index(event):
    index.value = 0
    axes_selected  = ''.join(event.new)
    if axes_selected == 'XY':
        index.end = (volume.shape[2]-1)
    elif axes_selected == 'XZ':
        index.end = (volume.shape[1]-1)
    elif axes_selected == 'YZ':
        index.end = (volume.shape[0]-1)

#### `watch` mechanism

In [ ]:
axes.param.watch(reset_index,['value'])

It keeps a watch on the axes parameters to adjust index values accordingly.

#### Dynamic Map

In [ ]:
dmap = hv.DynamicMap(lambda axes,index,cmap: get_brain_image(axes,index,cmap),
                     streams=[axes_stream,index_stream,cmap_stream])

#### Dashboard

In [ ]:
dashboard = pn.Column(
    pn.Column('# <pre> Brain Explorer',
              pn.Spacer(height=30)),
    pn.Row(
        pn.Row(dmap,
               pn.Spacer(width = 50),
               pn.Column(
                   pn.Spacer(height = 50),
                   cmaps,
                   pn.Spacer(height = 30),
                   axes,
                   pn.Spacer(height = 30),
                   index)
              )
        )
)

In [ ]:
dashboard.servable()

Deploy this dashboard from the CLI using:

```$ panel serve app.ipynb```